<a href="https://colab.research.google.com/github/Gabriele-Maggi/BWT_playground/blob/main/bwt_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [205]:
import numpy as np

input_string = "The quick brown fox jumps over the lazy dog"

input_string = "ABRACADABRA"

# Preprocessing iniziale del testo

In [206]:
def preprocess_string(input_string):
  lowercase_string = input_string.lower()
  underscored_string = lowercase_string.replace(" ", "_")
  final_string = underscored_string + "$"
  return final_string

preprocessed_string = preprocess_string(input_string)
print(preprocessed_string)  # Output: this_is_a_sample_string$

abracadabra$


# Funzioni di creazione dell'FM_INDEX

In [207]:
def get_alphabet(input_string):
  return sorted(list(set(input_string)))

def get_rotations(input_string, i):
  f = input_string[0:i]
  l = input_string[i:len(input_string)]
  return l+f

def order_by_lex(input_str):
  strings = [];
  for i in range(len(input_str)):
    strings.append(get_rotations(input_str, i))
  strings.sort()
  return strings

def get_bwt(lex_ordered_strings):
  return [s[-1] for s in lex_ordered_strings]

def get_occ(bwt):
  # ritrona un array (alfabeto x bwt)
  alph = get_alphabet(bwt);
  occ_lut = np.zeros((len(alph), len(bwt)))
  # array.index(letter)
  for i in range(len(alph)):
    for j in range(len(bwt)):
      if bwt[j] == alph[i]:
        occ_lut[i][j] = occ_lut[i][j-1] + 1
      else:
        occ_lut[i][j] = occ_lut[i][j-1]
  return occ_lut

def get_c(bwt):
  alph = get_alphabet(bwt);
  c = np.zeros(len(alph))
  sorted_bwt = sorted(bwt)
  for i in range(len(sorted_bwt)):
    ch = sorted_bwt[i];
    j = alph.index(ch)
    if c[j] == 0:
      c[j] = i
  return c


In [208]:
lex_ordered_strings = order_by_lex(preprocessed_string)
print(lex_ordered_strings)
bwt = get_bwt(lex_ordered_strings)
bwt


['$abracadabra', 'a$abracadabr', 'abra$abracad', 'abracadabra$', 'acadabra$abr', 'adabra$abrac', 'bra$abracada', 'bracadabra$a', 'cadabra$abra', 'dabra$abraca', 'ra$abracadab', 'racadabra$ab']


['a', 'r', 'd', '$', 'r', 'c', 'a', 'a', 'a', 'a', 'b', 'b']

# BWT utilizzi

## Ricostruzione del testo

In [232]:
occ = get_occ(bwt);
c = get_c(bwt);

# print(c)
# print(occ)
# print(bwt)

def recover_F(bwt, occ, c):
  n = len(bwt)
  alph = get_alphabet(bwt)
  t = ["" for x in range(n)]
  for i in range(n):
    sigma = alph.index(bwt[i])
    LF = occ[sigma][i] + c[sigma]
    LF = int(LF)
    print(LF)
    t[LF-1] = bwt[i]
  return t

def recover_t(bwt, occ, c):
  n = len(bwt)
  alph = get_alphabet(bwt)
  t = ["" for x in range(n)]

  last_lf = 0;
  for i in range(n):

      # print(last_lf)
      # print(bwt[last_lf])
      t[i] = bwt[last_lf]

      sigma = alph.index(bwt[last_lf])
      LF = int(occ[sigma][last_lf] + c[sigma])
      last_lf = LF-1;
  t = get_rotations(t, n-1)
  t = t[::-1]
  return t

# F = recover_F(bwt, occ, c)
# print(F)

t = recover_t(bwt, occ, c)
print(t)

['a', 'b', 'r', 'a', 'c', 'a', 'd', 'a', 'b', 'r', 'a', '$']


## Ricerca del pattern